In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    # plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    # plt.legend([string, 'val_'+string])
    plt.show()

In [3]:
df = pd.read_csv('Datasets/Raw_Dataset/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv')
df.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,15/02/2018 08:25:18,112641158,3,0,0,0,0,0,...,0,0.0,0.000000,0,0,56320579.0,7.042784e+02,56321077,56320081,Benign
1,22,6,15/02/2018 08:29:05,37366762,14,12,2168,2993,712,0,...,32,1024353.0,649038.754495,1601183,321569,11431221.0,3.644991e+06,15617415,8960247,Benign
2,47514,6,15/02/2018 08:29:42,543,2,0,64,0,64,0,...,32,0.0,0.000000,0,0,0.0,0.000000e+00,0,0,Benign
3,0,0,15/02/2018 08:28:07,112640703,3,0,0,0,0,0,...,0,0.0,0.000000,0,0,56320351.5,3.669884e+02,56320611,56320092,Benign
4,0,0,15/02/2018 08:30:56,112640874,3,0,0,0,0,0,...,0,0.0,0.000000,0,0,56320437.0,7.198347e+02,56320946,56319928,Benign


In [4]:
features = ['Fwd Seg Size Min', 'Init Fwd Win Byts', 'Fwd IAT Min', 'Bwd Pkt Len Max', 'Bwd IAT Mean', 'Pkt Len Max', 'Fwd IAT Tot', 'Fwd IAT Max', 'Bwd Pkt Len Std', 'Flow IAT Mean', 'ACK Flag Cnt', 'Fwd IAT Mean', 'Flow Duration', 'Flow IAT Min', 'Flow IAT Max']

In [5]:
def targetify(s):
    if s == 'Benign':
        return 0
    else:
        return 1

In [6]:
df['Target']=df['Label'].apply(targetify)

In [8]:
X = df[features]
X[features] = X[features].apply(pd.to_numeric, errors='coerce', axis=1)

C:\Users\young\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
y = df['Target']

In [10]:
X = X.fillna(0)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(838860, 15) (838860,)
(209715, 15) (209715,)


In [11]:
# # selecting 1% of random rows for better running time

# X_train2 = X_train.sample(frac=0.5, replace=True, random_state=1)
# y_train2 = y_train.sample(frac=0.5, replace=True, random_state=1)
# X_test2 = X_test.sample(frac=0.5, replace=True, random_state=1)
# y_test2 = y_test.sample(frac=0.5, replace=True, random_state=1)
# print (X_train2.shape, y_train2.shape)
# print( X_test2.shape, y_test2.shape)

(419430, 15) (419430,)
(104858, 15) (104858,)


In [12]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_train.values)
X_train = pd.DataFrame(x_scaled,columns=features)
x_scaled_test = min_max_scaler.fit_transform(X_test.values)
X_test = pd.DataFrame(x_scaled_test,columns=features)
# y_scaled = min_max_scaler.fit_transform(y_train2.values)
# y_train2 = pd.DataFrame(y_scaled,columns=features)
# y_scaled_test = min_max_scaler.fit_transform(y_test2.values)
# y_test2 = pd.DataFrame(y_scaled_test,columns=features)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(100000, 64), # since it doesn't consider "words," the embedding doesn't really matter
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    # tf.keras.layers.Dense(1, activation='softmax') # loss too big
])

In [14]:
model.compile(loss='binary_crossentropy',
              # optimizer='sgd', # almost same
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train.values, y_train.values, epochs=3)

Train on 838860 samples
Epoch 1/3
838860/838860 [==============================] - 2712s 3ms/sample - loss: 0.2000 - accuracy: 0.9499
Epoch 2/3
661408/838860 [======================>.......] - ETA: 9:30 - loss: 0.1984 - accuracy: 0.9499

In [16]:
# test_loss, test_acc = model.evaluate(X_test2.values, y_test2.values, batch_size=64)

# print('Test Loss: {}'.format(test_loss))
# print('Test Accuracy: {}'.format(test_acc))

In [17]:
pred_class = model.predict(X_test.values[:])
print(pred_class)

[[0.06088841]
 [0.04800892]
 [0.04800892]
 ...
 [0.04800896]
 [0.04800896]
 [0.06088838]]


In [18]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [19]:
print(round(pred_class[2][0]))
print(round(pred_class[1][0]))

0.0
0.0


In [20]:
predictions = [int(round(x[0])) for x in pred_class]
true_class = list(y_test)

In [21]:
print(true_class[:10], len(true_class))
print(predictions[:10], len(predictions))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 209715
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 209715


In [22]:
compdf = pd.DataFrame({'pred_class':predictions, 'true_class':true_class})
compdf = compdf.sort_values('pred_class', ascending=True)
print(str(compdf.head(10)))

        pred_class  true_class
0                0           0
139804           0           0
139805           0           0
139806           0           0
139807           0           0
139808           0           0
139809           0           0
139810           0           0
139811           0           0
139812           0           0


In [23]:
predictions = list(compdf['pred_class'].apply(myRound, r=225))
true_class = list(compdf['true_class'])

In [24]:
print(predictions[:10])
print(true_class[:10])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [25]:
from sklearn.metrics import confusion_matrix

In [26]:
confm = confusion_matrix(true_class, predictions)
confm

array([[199226,      0],
       [ 10489,      0]], dtype=int64)

In [27]:
tn, fp, fn, tp = confm.ravel()
(tn + tp)/(tn+tp+fn+fp)

0.9499845027775791